 # Wave Map Build and Export Final Models
 ## Model Descriptions
 There are two distinct models, one predicting the wave height and one predicting the direction.
 This approach is chosen because the wave height has a stronger correlation and this way the height prediction is better since it is not influenced by the direction.

 ### Wave Height Model
 **SGDRegressor with 'huber' loss**

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_height


 ### Wave Direction Model
 **RidgeRegression**
 Ridge is used because it can predict the entire direction vector as one value.

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_u
 * wave_v

In [1]:
import datetime
import pickle
import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

np.random.seed(42)


In [2]:
# Load data
wave_data = pd.read_csv('data/wave_data.csv', header=[0,1], index_col=0, parse_dates=True)
weather_data = pd.read_csv('data/weather_data.csv', header=[0,1], index_col=0, parse_dates=True)


In [3]:
# Join data
data = pd.concat([wave_data, weather_data], axis=1, join='inner')


In [4]:
# Add squared wind speed
sq_dat = data.loc[:,(slice(None),'wind_speed')]**2
sq_dat.columns = pd.MultiIndex.from_tuples([(b, p+'_sq') for b, p in sq_dat.columns])
data = pd.concat([data, sq_dat], axis=1)


In [5]:
# Add lagging values
lagging_vars = ['10u', '10v', '2t', 'sp', 'wind_speed', 'wind_speed_sq']
lagging_times = [datetime.timedelta(hours=h) for h in range(1, 20)]
dat = [data]
for var in lagging_vars:
    for i, t in enumerate(lagging_times):
        var_dat = data.loc[:,(slice(None),var)]
        var_dat.index = var_dat.index + t
        var_dat.columns = pd.MultiIndex.from_tuples([(b, p+'_lag'+str(i+1)) for b, p in var_dat.columns])
        dat.append(var_dat)
data = pd.concat(dat, axis=1)


In [6]:
# Helper function
def describe_regression(pipe, coef_desc, target_desc):
    model = pipe.named_steps['regression']
    if len(model.coef_.shape) <= 1:
        return pd.DataFrame(
            np.append(model.coef_, model.intercept_).T,
            index=np.append(coef_desc, 'intercept'), columns=target_desc)
    else:
        return pd.DataFrame(
            np.append(model.coef_, model.intercept_[:,np.newaxis], axis=1).T,
            index=np.append(coef_desc, 'intercept'), columns=target_desc)


In [7]:
# Another helper function
def plot_prediction(pipe, X, y, func=lambda x: x.flatten()):
    y_pred = pipe.predict(X)
    fig = px.scatter(x=func(y), y=func(y_pred))
    fig.update_layout(title='Prediction vs true value')
    fig.update_xaxes(title='true value')
    fig.update_yaxes(title='prediction')
    fig.show()


 ## Train Test Split
 Use a randomly chosen one of the buoys as the test dataset.

In [8]:
num_buoys = data.columns.levshape[0]
test_buoy = 'pohjois-itaemeri'


In [9]:
test = data[test_buoy]
train = data.drop(test_buoy, axis=1)


 ## Wave Height Model

In [10]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['Wave height (m)']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [11]:
# Train model
height_pipe = Pipeline([('scaler', StandardScaler()), ('regression', SGDRegressor(loss='huber'))])
height_pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(height_pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.763


In [12]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(height_pipe.score(X_test, Y_test)))
plot_prediction(height_pipe, X_test, Y_test)


Test accuracy of the model: 0.737


In [13]:
describe_regression(height_pipe, input_vars, output_vars)


,Wave height (m)
10u,-0.012306
10u_lag2,-0.000770
10u_lag4,0.033970
10v,-0.007724
10v_lag2,0.014105
10v_lag4,-0.022226
wind_speed,-0.048085
wind_speed_lag2,-0.088722
wind_speed_lag4,-0.005834
wind_speed_sq,0.267644


In [14]:
# Save model
with open('models/wave-height-model.pkl', 'wb') as f:
    pickle.dump(height_pipe, f)


 ## Wave Direction Model

In [15]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [16]:
# Train model
alphas = np.logspace(-2, 5, num=20)
dir_pipe = Pipeline([('scaler', StandardScaler()), ('regression', RidgeCV())])
dir_pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(dir_pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(dir_pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=10.000
Train accuracy of the model: 0.659


In [17]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(dir_pipe.score(X_test, Y_test)))
plot_prediction(dir_pipe, X_test, Y_test, func=lambda x: np.sqrt(np.sum(x**2, axis=1)))


Test accuracy of the model: 0.627


In [18]:
# Model coefficiens
describe_regression(dir_pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.132618,0.102788
10u_lag2,0.051085,-0.120780
10u_lag4,0.295503,0.126620
10v,-0.016676,0.132647
10v_lag2,0.042303,0.071433
10v_lag4,-0.015191,0.361553
wind_speed,-0.096793,-0.028199
wind_speed_lag2,-0.067065,0.026061
wind_speed_lag4,-0.060660,0.056297
wind_speed_sq,0.157682,-0.003454


In [19]:
# Save model
with open('models/wave-dir-model.pkl', 'wb') as f:
    pickle.dump(dir_pipe, f)


 ## Test Performance of Combined Model

In [20]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values


In [21]:
# Combine predictions
height = height_pipe.predict(X_test)
dir = dir_pipe.predict(X_test)

scale_factor = height / np.sqrt((dir**2).sum(axis=1))

pred = dir * np.concatenate([scale_factor[:,np.newaxis], scale_factor[:,np.newaxis]], axis=1)


In [22]:
# Test performance
print('Test accuracy of the combined models: {:.3f}'.format(r2_score(Y_test, pred)))

fig = px.scatter(x=np.sqrt(np.sum(Y_test**2, axis=1)), y=np.sqrt(np.sum(pred**2, axis=1)))
fig.update_layout(title='Prediction vs true value')
fig.update_xaxes(title='true value')
fig.update_yaxes(title='prediction')
fig.show()

Test accuracy of the combined models: 0.726
